In [1]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Importowanie danych
chess_king_rook_vs_king = fetch_ucirepo(id=23)
X = chess_king_rook_vs_king.data.features
y = chess_king_rook_vs_king.data.targets
df_chess = pd.concat([X, y], axis=1)

## Mapowanie ramki danych zgodnie z poleceniem

In [2]:
df_mapped = df_chess.copy()
df_mapped['white-depth-of-win'] = df_chess['white-depth-of-win'].replace({'draw' : -1, 'zero' : 0, 'one' : 1, 'two' : 2, 'three' : 3, 'four' : 4, 'five' : 5, 'six' : 6, 'seven' : 7, 'eight' : 8, 'nine' : 9, 'ten' : 10, 'eleven' : 11, 'twelve' : 12, 'thirteen' : 13, 'fourteen' : 14, 'fifteen' : 15, 'sixteen' : 16})

def map_letters(x):
    if x == 'a':
        return 1
    elif x == 'b':
        return 2
    elif x == 'c':
        return 3
    elif x == 'd':
        return 4
    elif x == 'e':
        return 5
    elif x == 'f':
        return 6
    elif x == 'g':
        return 7
    elif x == 'h':
        return 8
    return np.nan

df_mapped['white-king-file-numerous'] = df_mapped['white-king-file'].apply(map_letters)
df_mapped['white-rook-file-numerous'] = df_mapped['white-rook-file'].apply(map_letters)
df_mapped['black-king-file-numerous'] = df_mapped['black-king-file'].apply(map_letters)

df_mapped['euclidean_metric'] = np.sqrt(np.square(df_mapped['white-king-rank'] - df_mapped['black-king-rank']) + np.square(df_mapped['white-king-file-numerous'] - df_mapped['black-king-file-numerous']))

df_mapped['manhattan_metric'] = np.abs(df_mapped['white-king-rank'] - df_mapped['black-king-rank']) + np.abs(df_mapped['black-king-file-numerous'] - df_mapped['white-king-file-numerous'] )

df_mapped['czebyszew_metric'] = np.maximum(np.abs(df_mapped['white-king-rank'] - df_mapped['black-king-rank']), np.abs(df_mapped['black-king-file-numerous'] - df_mapped['white-king-file-numerous']))

def dist_from_edge(x):
    if x == 1:
        return 0
    elif x == 2:
        return 1
    elif x == 3:
        return 2
    elif x == 4:
        return 3
    elif x == 5:
        return 3
    elif x == 6:
        return 2
    elif x == 7:
        return 1
    elif x == 8:
        return 0
    return np.nan

df_mapped['dist_from_edge_fill'] = df_mapped['black-king-file-numerous'].apply(dist_from_edge)
df_mapped['dist_from_edge_rank'] = df_mapped['black-king-rank'].apply(dist_from_edge)

df_mapped['black_closer_edge_dist'] = np.minimum(df_mapped['dist_from_edge_fill'], df_mapped['dist_from_edge_rank'])

/var/folders/h7/16x3tz8d4mxd7h0_p4b1qjy00000gn/T/ipykernel_66700/370120019.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_mapped['white-depth-of-win'] = df_chess['white-depth-of-win'].replace({'draw' : -1, 'zero' : 0, 'one' : 1, 'two' : 2, 'three' : 3, 'four' : 4, 'five' : 5, 'six' : 6, 'seven' : 7, 'eight' : 8, 'nine' : 9, 'ten' : 10, 'eleven' : 11, 'twelve' : 12, 'thirteen' : 13, 'fourteen' : 14, 'fifteen' : 15, 'sixteen' : 16})


---

# ZADANIE 1

In [3]:
# Przygotowanie ramek pod rysowanie heatmapy

df_mat = df_mapped.loc[df_mapped['white-depth-of-win'] == 0, ["black-king-file-numerous", 'black-king-rank']]
df_mat_heatmap = df_mat.pivot_table(
    index = 'black-king-rank',
    columns = 'black-king-file-numerous',
    aggfunc = 'size',
    fill_value = 0
).reindex(
    index = range(1, 9),
    columns = range(1, 9),
    fill_value = 0
)

df_draw = df_mapped.loc[df_mapped['white-depth-of-win'] == -1, ["black-king-file-numerous", 'black-king-rank']]
df_draw_heatmap = df_draw.pivot_table(
    index = 'black-king-rank',
    columns = 'black-king-file-numerous',
    aggfunc = 'size',
    fill_value = 0
).reindex(
    index = range(1, 9),
    columns = range(1, 9),
    fill_value = 0
)

In [4]:
# Funkcja do rysowania heatmapy

def draw_heatmap(df, title, colorscale):
    fig = px.imshow(df,
          origin='lower',
          text_auto=True,
          color_continuous_scale=colorscale,)

    fig.update_traces(
        xgap=1,
        ygap=1
    )
    fig.update_layout(
        title = title,
        xaxis=dict(
            dtick=1,
            title='numer kolumny'
        ),
        yaxis=dict(
            dtick=1,
            title='numer wiersza'
        ),
        coloraxis_colorbar=dict(
            title='Liczba wystąpień'
        ),
    )
    return fig

In [5]:
draw_heatmap(df_mat_heatmap, 'Liczba wystąpień pozycji czarnego króla, kiedy jest matowany', 'Blues').show()

### Zgodnie z powyższą heatmapą widzimy, że gdy król czarny jest matowany, to król czarny najczęściej znajduje się w pierwszej kolumnie i pierwszym wierszu, co odpowiada szachowej pozycji a1

In [6]:
draw_heatmap(df_draw_heatmap, 'Liczba wystąpień pozycji czarnego króla, kiedy jest remis', 'Greens').show()

### Zgodnie z powyższą heatmapą widzimy, że gdy gra kończy się remisem, to król czarny najczęściej znajduje się po prawej stronie (dokładniej w kolumnie 6/7) czyli na pozycjach: g2, g3, g4, g5, g6

---

## ZADANIE 2

In [7]:
# Ignorujemy sytuacje remisowe

df_mapped_clean = df_mapped.loc[df_mapped['white-depth-of-win'] != -1, :].copy()

min_val = np.floor(df_mapped_clean['euclidean_metric'].min())
max_val = np.ceil(df_mapped_clean['euclidean_metric'].max())

bins_edges = np.arange(min_val, max_val + 1, 1)
df_mapped_clean['euclidean_metric_binned'] = pd.cut(df_mapped_clean['euclidean_metric'], bins=bins_edges, right=False)
df_mapped_clean['euclidean_metric_binned_str'] = df_mapped_clean['euclidean_metric_binned'].astype(str)
df_mapped_clean_sorted = df_mapped_clean.sort_values(by='euclidean_metric_binned_str')

In [8]:
fig2 = make_subplots(rows = 3, cols = 1,
                    subplot_titles = ('Metryka Czebyszewa', 'Metryka Manhattan', 'Metryka Euklidesowa'))

fig2.add_trace(go.Box(x = df_mapped_clean['czebyszew_metric'],
                     y = df_mapped_clean['white-depth-of-win'],
                     marker=dict(color='lightgreen')),
              row=1, col=1)

fig2.add_trace(go.Box(x = df_mapped_clean['manhattan_metric'],
                     y = df_mapped_clean['white-depth-of-win'],
                     marker=dict(color='lightgreen')),
              row=2, col=1)

fig2.add_trace(go.Box(x = df_mapped_clean_sorted['euclidean_metric_binned_str'],
                     y = df_mapped_clean_sorted['white-depth-of-win'],
                     marker=dict(color='lightgreen')),
              row=3, col=1)

fig2.update_xaxes(title_text='Odległość między królami liczona za pomocą metryki Czebyszewa', row=1, col=1)
fig2.update_yaxes(title_text='Liczba ruchów do zakończenia gry', row=1, col=1)

fig2.update_xaxes(title_text='Odległość między królami liczona za pomocą metryki Manhattan', row=2, col=1)
fig2.update_yaxes(title_text='Liczba ruchów do zakończenia gry', row=2, col=1)

fig2.update_xaxes(title_text='Odległość między królami liczona za pomocą metryki Euklidesowej', row=3, col=1)
fig2.update_yaxes(title_text='Liczba ruchów do zakończenia gry', row=3, col=1)

fig2.update_layout(
    height=1500,
    showlegend=False,
)

### Która metryka jest najlepsza?
#### Na pierwszy rzut oka najgorzej wypada metryka Manhatan; Mimo, że w każdej z metryk widać trend, że wraz ze wzrotem odległości zwiększa się liczba potrzebnych do skończenia gry, to przy metryce Manhatan ten trend się najbardziej wypłaszcza.
#### W wyborze metryki Czebyszewa nad metryką Euklidesową przemawia fakt, że żeby wykers metryki Euklidesowej był jakkolwiek czytelny musieliśmy podzielić wartości na kubełki, a w metryce Czebyszewa mamy "kubełki" odpowiadające liczbom całkowitym

---

## Zadanie 3

In [32]:
fig3 = px.box(
    df_mapped_clean,
    x='black_closer_edge_dist',
    y='white-depth-of-win',
    title='Wpływ odległości czarnego króla na długość gry',
    labels={
        'black_closer_edge_dist': 'Odległość Czarnego Króla od krawędzi',
        'white-depth-of-win': 'Liczba ruchów do wygranej'
    },
)
fig3.update_layout(showlegend=False)

In [30]:
heatmap_data = df_mapped_clean.pivot_table(index='black_closer_edge_dist', columns='czebyszew_metric', values='white-depth-of-win', aggfunc='mean')
heatmap_data = heatmap_data.sort_index(ascending=True)

fig_4 = px.imshow(
    heatmap_data,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    text_auto=True,
    origin='lower',
    color_continuous_scale='Greens',
)

fig_4.update_layout(
        title = 'Średnia liczba ruchów w zależności od obu dystansów',
        xaxis=dict(
            title='Odległość między królami (Czebyszew)'
        ),
        yaxis=dict(
            dtick=1,
            title='Odległość czarnego króla od krawędzi'
        ),
        coloraxis_colorbar=dict(
            title='Liczba ruchów do skończenia gry'
        )
)
fig_4.update_yaxes(tickvals=[0, 1, 2, 3], ticktext=["0", "1", "2", "3"])

fig_4.show()

### Czy informacja o odległości czarnego króla od krawędzi planszy może być przydatna w określeniu w ilu ruchach zakończy się gra? Czy uzupełnienie tej informacji o odległość pomiędzy białym a czarnym królem z poprzedniego zadania może dodatkowo pomóc?

Na podstawie pierwszego widzimy wyraźnie, że odległość czarrnego króla od krawędzi planmszy jest przydatna w określeniu w ilu ruchach zakończy się gra, ponieważ widzimy, że wraz ze wzrostem odległości zwiększa się także liczba ruchów potrzebnych do wygranej. Widzimy dzięki temu, że gry kończą się szybciej, gdy król jest przy krawędziach.

Jak widzimy na drugim wykresie dodanie informacji o odległości pomiędzy białym, a czarnym królem (w metryce Czebyszewa) zwiększa precyzję w szacowaniu. Np. widzimy, że gdy król jest blisko krawędzi ma średnio 11 ruchów do zakończenia gry (box plot), jednak po dodaniu informacji o odległości między królami poprawia nam precyzję i dzięki temu widzimy, że gdy odległość między królami jest równa 2, a król przy krawędzi to wystarczy średnio między 6 a 7 ruchów do zakończenia partii.